## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**3230. Customer Purchasing Behavior Analysis (Medium)**

**Table: Transactions**

| Column Name      | Type    |
|------------------|---------|
| transaction_id   | int     |
| customer_id      | int     |
| product_id       | int     |
| transaction_date | date    |
| amount           | decimal |

transaction_id is the unique identifier for this table.
Each row of this table contains information about a transaction, including the customer ID, product ID, date, and amount spent.

**Table: Products**

| Column Name | Type    |
|-------------|---------|
| product_id  | int     |
| category    | varchar |
| price       | decimal |

product_id is the unique identifier for this table.
Each row of this table contains information about a product, including its category and price.

**Write a solution to analyze customer purchasing behavior. For each customer, calculate:**
- The total amount spent.
- The number of transactions.
- The number of unique product categories purchased.
- The average amount spent. 
- The most frequently purchased product category (if there is a tie, choose the one with the most recent transaction).
- A loyalty score defined as: (Number of transactions * 10) + (Total amount spent / 100).

Round total_amount, avg_transaction_amount, and loyalty_score to 2 decimal places.

Return the result table ordered by loyalty_score in descending order, then by customer_id in ascending order.

The query result format is in the following example.

**Example:**

**Input:**

**Transactions table:**

| transaction_id | customer_id | product_id | transaction_date | amount |
|----------------|-------------|------------|------------------|--------|
| 1              | 101         | 1          | 2023-01-01       | 100.00 |
| 2              | 101         | 2          | 2023-01-15       | 150.00 |
| 3              | 102         | 1          | 2023-01-01       | 100.00 |
| 4              | 102         | 3          | 2023-01-22       | 200.00 |
| 5              | 101         | 3          | 2023-02-10       | 200.00 |

**Products table:**

| product_id | category | price  |
|------------|----------|--------|
| 1          | A        | 100.00 |
| 2          | B        | 150.00 |
| 3          | C        | 200.00 |

**Output:**

| customer_id | total_amount | transaction_count | unique_categories | avg_transaction_amount | top_category | loyalty_score |
|-------------|--------------|-------------------|-------------------|------------------------|--------------|---------------|
| 101         | 450.00       | 3                 | 3                 | 150.00                 | C            | 34.50         |
| 102         | 300.00       | 2                 | 2                 | 150.00                 | C            | 23.00         |

**Explanation:**
- For customer 101:
  - Total amount spent: 100.00 + 150.00 + 200.00 = 450.00
  - Number of transactions: 3
  - Unique categories: A, B, C (3 categories)
  - Average transaction amount: 450.00 / 3 = 150.00
  - Top category: C (Customer 101 made 1 purchase each in categories A, B, and C. Since the count is the same for all categories, we choose the most recent transaction, which is category C on 2023-02-10)
  - Loyalty score: (3 * 10) + (450.00 / 100) = 34.50

- For customer 102:
  - Total amount spent: 100.00 + 200.00 = 300.00
  - Number of transactions: 2
  - Unique categories: A, C (2 categories)
  - Average transaction amount: 300.00 / 2 = 150.00
  - Top category: C (Customer 102 made 1 purchase each in categories A and C. Since the count is the same for both categories, we choose the most recent transaction, which is category C on 2023-01-22)
  - Loyalty score: (2 * 10) + (300.00 / 100) = 23.00

**Note:** The output is ordered by loyalty_score in descending order, then by customer_id in ascending order.


In [0]:
transactions_data_3230 = [
    (1, 101, 1, "2023-01-01", 100.00),
    (2, 101, 2, "2023-01-15", 150.00),
    (3, 102, 1, "2023-01-01", 100.00),
    (4, 102, 3, "2023-01-22", 200.00),
    (5, 101, 3, "2023-02-10", 200.00)
]

transactions_columns_3230 = ["transaction_id", "customer_id", "product_id", "transaction_date", "amount"]
transactions_df_3230 = spark.createDataFrame(transactions_data_3230, transactions_columns_3230)
transactions_df_3230.show()

products_data_3230 = [
    (1, "A", 100.00),
    (2, "B", 150.00),
    (3, "C", 200.00)
]

products_columns_3230 = ["product_id", "category", "price"]
products_df_3230 = spark.createDataFrame(products_data_3230, products_columns_3230)
products_df_3230.show()

+--------------+-----------+----------+----------------+------+
|transaction_id|customer_id|product_id|transaction_date|amount|
+--------------+-----------+----------+----------------+------+
|             1|        101|         1|      2023-01-01| 100.0|
|             2|        101|         2|      2023-01-15| 150.0|
|             3|        102|         1|      2023-01-01| 100.0|
|             4|        102|         3|      2023-01-22| 200.0|
|             5|        101|         3|      2023-02-10| 200.0|
+--------------+-----------+----------+----------------+------+

+----------+--------+-----+
|product_id|category|price|
+----------+--------+-----+
|         1|       A|100.0|
|         2|       B|150.0|
|         3|       C|200.0|
+----------+--------+-----+



In [0]:
trans_with_cat_3230 = transactions_df_3230\
                            .join(products_df_3230, on="product_id", how="left")

In [0]:
agg_metrics_3230 = trans_with_cat_3230\
                        .groupBy("customer_id")\
                            .agg(
                                round(sum("amount"), 2).alias("total_amount"),
                                count("transaction_id").alias("transaction_count"),
                                countDistinct("category").alias("unique_categories"),
                                round(avg("amount"), 2).alias("avg_transaction_amount")
                                )

In [0]:
window_spec = Window.partitionBy("customer_id").orderBy(col("category_count").desc(), col("latest_date").desc())

In [0]:
cat_stats_3230 = trans_with_cat_3230\
                    .groupBy("customer_id", "category")\
                        .agg(
                            count("transaction_id").alias("category_count"),
                            sum("amount").alias("total_category_amount"),
                            sum("amount").alias("latest_date")
                            )

In [0]:
cat_stats_3230 = trans_with_cat_3230\
                    .groupBy("customer_id", "category")\
                        .agg(
                            count("transaction_id").alias("category_count"),
                            sum("amount").alias("total_category_amount"),
                            sum(col("transaction_date")).alias("latest_date")
                            )

In [0]:
cat_stats_3230 = trans_with_cat_3230\
                    .groupBy("customer_id", "category")\
                        .agg(
                            count("transaction_id").alias("category_count"),
                            max("transaction_date").alias("latest_date")
                            )

In [0]:
top_cat_ranked_3230 = cat_stats_3230\
                        .withColumn("rank", row_number().over(window_spec))

In [0]:
top_category_3230 = top_cat_ranked_3230\
                        .filter(col("rank") == 1).select("customer_id", col("category").alias("top_category"))

In [0]:
agg_metrics_3230\
    .join(top_category_3230, on="customer_id", how="left")\
        .withColumn("loyalty_score", round(col("transaction_count") * 10 + col("total_amount") / 100, 2))\
            .orderBy(col("loyalty_score").desc(), col("customer_id").asc()).display()

customer_id,total_amount,transaction_count,unique_categories,avg_transaction_amount,top_category,loyalty_score
101,450.0,3,3,150.0,C,34.5
102,300.0,2,2,150.0,C,23.0
